In [ ]:
# imports
import torch
import math
import copy
import torch.optim as optim
import torch.nn as nn

import numpy as np
import matplotlib.pyplot as plt

from dataset import TrafficSignDataset
from torch.utils.data import Subset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from matplotlib.patches import Rectangle
from tqdm import tqdm
from datetime import datetime

from networks import TrafficSignsClassifier

In [ ]:
from preprocessing import get_transforms
# transform = None
transform = get_transforms(img_size=32)

In [ ]:
dataset = TrafficSignDataset("..\data\Train.csv", "..\data", transform=transform)

# randomly split into train and validation, regardless of frame sequences
train, val = random_split(dataset, [math.floor(len(dataset) * 0.9), math.ceil(len(dataset) * 0.1)], generator=torch.Generator().manual_seed(42))

print(len(train), len(val))

In [ ]:
cols, rows = 5, 6
idx = 0 # index of frame sequence

figure = plt.figure(figsize=(20, 24))

for i in range(idx * 30, idx * 30 + 30):
    img, target = train[i]

    figure.add_subplot(cols, rows, i - (idx * 30) + 1)
    plt.imshow(img.byte().permute(1,2,0))
    ax = plt.gca()

    bbox = target["bbox"]
    x = bbox[2]
    y = bbox[3]
    box_width = bbox[4] - x
    box_height = bbox[5] - y

    rect = Rectangle((x,y), box_width, box_height, linewidth=1, edgecolor='r',facecolor='none')

    plt.axis("off")
    ax.add_patch(rect)

plt.show()

In [ ]:
fig, ax = plt.subplots(3,1, figsize=(15,15))

# training data
train_labels, train_counts = np.unique((train.dataset.targets["ClassId"])[train.indices], return_counts=True)
ax[0].bar(train_labels, train_counts / len(train) * 100)

# validation data
val_labels, val_counts = np.unique((val.dataset.targets["ClassId"])[val.indices], return_counts=True)
ax[1].bar(val_labels, val_counts, color="orange")


# val - train
diff = val_counts / len(val) - train_counts / len(train)

ax[2].bar(train_labels, diff * 100, color=np.where(diff >= 0, "orange","C0"))


In [ ]:
# setup dataloaders
train_dataloader = DataLoader(train, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val, batch_size=32, shuffle=False)
# TODO: test_dataloader

# to test training setup for errors
sample_subset = Subset(dataset, np.arange(640))
sample_dataloader =  DataLoader(sample_subset, batch_size=32, shuffle=True)

In [ ]:
# setup device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

In [ ]:
hparams = {
    "lr": 1e-3
}

In [ ]:
# init model
model = TrafficSignsClassifier(hparams, input_size=32, num_classes=len(train_labels))

model = model.to(device)

In [ ]:
# define optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=hparams["lr"])

In [ ]:
# new tensorboard log
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = SummaryWriter(f"../runs/1_layer_linear_clf_{timestamp}")

correct = 0.0
total = 0
max_epochs = 20
validate_every = 1

min_val_loss = None
patience = 0
stopping_threshold = 5

for epoch in range(max_epochs):
    running_loss = 0.0

    for i, data in enumerate(tqdm(train_dataloader)):
        # move data to device
        x, y = data
        x = x.to(device)
        y = y["label"].to(device)

        # zero param gradients
        optimizer.zero_grad()

        # forward + backward + optimizer
        logits = model(x)
        loss = criterion(logits, y)
        loss.backward()
        optimizer.step()
        
        # calc stats
        running_loss += loss.item()
        _, preds = torch.max(logits, 1)
        correct += preds.eq(y).sum().item()
        total += y.size(0)
        
    # print stats to console
    running_loss /= len(train_dataloader)
    correct /= total
    print("[Epoch %d/%d] loss: %.3f acc: %.2f %%" % (epoch+1, max_epochs, running_loss, 100*correct))

    # log to tensorboard
    writer.add_scalar('Training loss', running_loss, epoch * len(train_dataloader) + i)
    writer.add_scalar('Training acc', correct, epoch * len(train_dataloader))

    # reset stats after epoch
    running_loss = 0.0
    correct = 0.0
    total = 0

    # validation loop
    if epoch % validate_every == (validate_every - 1):

        val_running_loss = 0.0
        val_acc = 0.0
        val_total = 0

        with torch.no_grad():
            for i, data in enumerate(val_dataloader):
                x, y = data
                x, y = x.to(device), y["label"].to(device)
                
                logits = model(x)
                val_loss = criterion(logits, y)
                val_running_loss += val_loss.item()

                _, preds = torch.max(logits, 1)
                val_acc += preds.eq(y).sum().item()
                val_total += y.size(0)


        val_running_loss /= len(val_dataloader)
        val_acc /= val_total
        print("Validation loss: %.3f acc: %.2f" % (val_running_loss, val_acc * 100))

        # log to tensorboard
        writer.add_scalar('Validation loss', val_running_loss, epoch * len(train_dataloader) + i)
        writer.add_scalar('Validation acc', val_acc, epoch * len(train_dataloader))
    
        # save best model and early stopping
        if min_val_loss is None or val_running_loss < min_val_loss:
            patience = 0
            min_val_loss = val_running_loss
            best_acc = val_acc

            # save params
            best_model_weights = copy.deepcopy(model.state_dict())
        elif patience >= stopping_threshold:
            print("Early stopping, best model at epoch ", epoch - stopping_threshold)
            break
        else:
            patience += 1
            continue

# load best weights
model.load_state_dict(best_model_weights)

### Confusion matrix

In [ ]:
num_classes = len(train_labels)

confusion_matrix = torch.zeros(num_classes, num_classes)
with torch.no_grad():
    for i, (x, y) in enumerate(val_dataloader):
        x = x.to(device)
        y = y["label"].to(device)
        outputs = model(x)
        _, preds = torch.max(outputs, 1)
        for t, p in zip(y.view(-1), preds.view(-1)):
                confusion_matrix[t, p] += 1

print(confusion_matrix)

In [ ]:
import seaborn as sns

rows_sum = confusion_matrix.sum(axis=1)

norm_confusion_matrix = confusion_matrix / rows_sum[:, np.newaxis]

sns.set(rc={'figure.figsize':(24,20)})
ax = sns.heatmap(norm_confusion_matrix, annot=True, cmap='Blues', fmt='.1f')

plt.show()